<a href="https://colab.research.google.com/github/Devica2000/StanfordTech16LLM/blob/main/Devica_Verma_Tech16_HW4_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index-embeddings-openai
!pip install llama-index-llms-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.3 MB/s eta 0:00:00


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = open_ai_key

In [ ]:
!pip install pypdf

In [ ]:
from llama_index.core import SimpleDirectoryReader

In [ ]:
from llama_index.core import Settings

In [ ]:
from llama_index.core import StorageContext

In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core import VectorStoreIndex


In [ ]:
from llama_index.core.tools import QueryEngineTool

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)

# Extra Credit: Build a Routing Agent to answer questions related to class video lectures and provided notes

* Used the youtube-transcript-api to load lecture videos of the two courses (CS229: Machine Learning and CS224N: NLP with Deep Learning at Stanford).
* Used SimpleDirectoryReader to load lectures notes (available as PDF's online).
* Created separate nodes for each of the video lecture and notes for the two courses. Then, combined the video and lecture notes node of a course into a single combined node for that course.
* Build summary tool and vector tool for each of the courses
* When asked a question, the routing agent would identify which tool to use to answer the question

Note: I have only used a lecture video and notes pdf from a single class for each of the two courses.

The idea is to be able to extend this functionality to all the course content available to a student for a specific course. When enrolled in a course, there are multiple sources of information. For example, lecture slides, class notes, class video lecture, extra reading papers, Ed Stem, Office Hours etc. Instead of the student having to go through all the course content, the agent should be able to direct them to the correct source(s) for the question being asked. This would streamline the process and would allow to save the time it takes to just find out where a particular topic/question was mentioned in the different types of course materials.


In [ ]:
!pip install youtube-transcript-api

In [ ]:
!pip install llama-index-readers-youtube-transcript
!pip install llama-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 26.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 22.6 MB/s eta 0:00:00
  Created wheel for llama-core: filename=llama_core-0.3.3-cp310-cp310-linux_x86_64.whl size=3111539 sha256=49c8c724176f858972d4303c5afd02f929dfb74525a11de004e93ca6464cb046
  Stored in directory: /root/.cache/pip/wheels/8e/0b/e3/3437eccf6611f82c299e6380dfb08adbfb2ce06a2da2d8e4bb
Successfully built llama-core


In [ ]:
# Load the transcript
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()

# Load the first video lecture CS229
video_transcript_ml = loader.load_data(
    ytlinks=["https://www.youtube.com/watch?v=wr9gUr-eWdA"]
)
print(video_transcript_ml[0])



Doc ID: wr9gUr-eWdA
Text: Hello everyone. Uh, so my name is Raphael Townshend. I'm one of
the head TAs for this class. This week Andrew is traveling and my
advisor is still dealing with medical issues. So I'm going to be
giving today's lecture. Um, you heard from my wonderful co-head TA
Anand a couple of weeks ago. And so today, we're gonna be going over
decision trees a...


In [ ]:
# Load the first video lecture for CS224N

video_transcript_nlp = loader.load_data(
    ytlinks=["https://www.youtube.com/watch?v=rmVRLeJRkl4"]
)
print(video_transcript_nlp[0])

Doc ID: rmVRLeJRkl4
Text: um hi everybody um welcome to stanford cs224 n also known as
ling 284 natural language processing with deep learning i'm
christopher manning and i'm the main instructor for this class so what
we hope to do today is to dive right in so i'm going to spend about 10
minutes talking about the course and then we're going to get straight
into content f...


In [ ]:
# loading the notes pdf for each of these classes

from llama_index.core import SimpleDirectoryReader

!wget https://cs229.stanford.edu/cs229-notes-decision_trees.pdf
!wget https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1244/readings/cs224n_winter2023_lecture1_notes_draft.pdf

--2024-08-07 23:06:15--  https://cs229.stanford.edu/cs229-notes-decision_trees.pdf
Resolving cs229.stanford.edu (cs229.stanford.edu)... 171.64.64.64
Connecting to cs229.stanford.edu (cs229.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 242394 (237K) [application/pdf]
Saving to: ‘cs229-notes-decision_trees.pdf’

cs229-notes-decisio 100%[===================>] 236.71K  --.-KB/s    in 0.08s   

2024-08-07 23:06:15 (3.07 MB/s) - ‘cs229-notes-decision_trees.pdf’ saved [242394/242394]

--2024-08-07 23:06:15--  https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1244/readings/cs224n_winter2023_lecture1_notes_draft.pdf
Resolving web.stanford.edu (web.stanford.edu)... 171.67.215.200, 2607:f6d0:0:925a::ab43:d7c8
Connecting to web.stanford.edu (web.stanford.edu)|171.67.215.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 216720 (212K) [application/pdf]
Saving to: ‘cs224n_winter2023_lecture1_notes_draft.pdf’

cs2

In [ ]:
notes_ml = SimpleDirectoryReader("./").load_data("cs229-noted-decision_trees.pdf")
notes_nlp = SimpleDirectoryReader("./").load_data("cs224n_winter2023_lecture1_notes_draft.pdf")


Loading files: 100%|██████████| 3/3 [00:01<00:00,  2.08file/s]


In [ ]:
# notes_nlp

In [ ]:
from llama_index.core import Settings

# initialize settings (set chunk size)
Settings.chunk_size = 1024

# Creating nodes for all of them
video_nodes_ml = Settings.node_parser.get_nodes_from_documents(video_transcript_ml)
video_nodes_nlp = Settings.node_parser.get_nodes_from_documents(video_transcript_nlp)

notes_nodes_ml = Settings.node_parser.get_nodes_from_documents(notes_ml)
notes_nodes_nlp = Settings.node_parser.get_nodes_from_documents(notes_nlp)

In [ ]:
# Combine nodes for each course
all_nodes_ml = video_nodes_ml + notes_nodes_ml
all_nodes_nlp = video_nodes_nlp + notes_nodes_nlp

In [ ]:
summary_index_ml = SummaryIndex(all_nodes_ml)
vector_index_ml = VectorStoreIndex(all_nodes_ml)

summary_index_nlp = SummaryIndex(all_nodes_nlp)
vector_index_nlp = VectorStoreIndex(all_nodes_nlp)

In [ ]:
summary_query_engine_ml = summary_index_ml.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
    verbose = True
)
vector_query_engine_ml = vector_index_ml.as_query_engine(verbose = True)


In [ ]:
summary_query_engine_nlp = summary_index_nlp.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
    verbose = True
)
vector_query_engine_nlp = vector_index_nlp.as_query_engine(verbose = True)

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)
from llama_index.core.tools import QueryEngineTool


summary_tool_ml = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine_ml,
    description="Only useful to summarize content related to the data source about Decision Trees",
)

vector_tool_ml = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine_ml,
    description="Useful for retrieving specific context related to the data source about Decision Trees",
)

summary_tool_nlp = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine_nlp,
    description="Only Useful for summarization questions related to the data source which talks about Natural Language Processing (NLP)",
)

vector_tool_nlp = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine_nlp,
    description="Useful for retrieving specific context related to the data source which talks about Natural Language Processing (NLP)",
)

query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool_ml,
        vector_tool_ml,
        summary_tool_nlp,
        vector_tool_nlp
    ],
    verbose = True
)


In [ ]:
response = query_engine.query("What is the formula for gain?")
print(response)

Selecting query engine 0: The question is about a specific concept (formula for gain) related to Decision Trees, which is summarized in choice (1)..
The formula for gain is typically represented as G(j, θ) = C(S) - (|L|/|S|)C(L) - (|R|/|S|)C(R).


In [ ]:
response = query_engine.query("Why do we need XGBoost?")
print(response)

Selecting query engine 1: XGBoost is a specific context related to the data source about Decision Trees.
We need XGBoost because it is a state-of-the-art algorithm that includes enhancements such as optimizing the loss function with a regularization term, using a subset of features when splitting tree nodes, and employing a second-order approximation in the boosting algorithm. These features make XGBoost a powerful tool for machine learning tasks and it is commonly used in industry settings.


In [ ]:
response = query_engine.query("Why do we need word vectors?")
print(response)

Selecting query engine 3: Word vectors are useful for retrieving specific context related to the data source which talks about Natural Language Processing (NLP).
To better compute with words.


In [ ]:
response = query_engine.query("What is one hot encoding?")
print(response)

Selecting query engine 1: One hot encoding is a specific context related to the data source about Decision Trees..
The one hot encoding is a method where a word is represented as a vector with all zeros except for one element which is set to one, indicating the presence of that word in a specific position within the vector.


In [ ]:
response = query_engine.query("What is one hot encoding in a corpus?")
print(response)

Selecting query engine 3: The question is related to Natural Language Processing (NLP) and specifically asks about one hot encoding in a corpus, which is a specific context related to NLP..
The one hot encoding in a corpus refers to representing a word as a vector where all elements are zero except for the element corresponding to the word, which is set to one.


In [ ]:
response = query_engine.query("Summarize the advantages of XGBoost over Random Forest in 4 bullet points.")
print(response)

Selecting query engine 1: The question requires retrieving specific context related to the data source about Decision Trees, which is relevant to summarizing the advantages of XGBoost over Random Forest..
- XGBoost optimizes the loss function with a regularization term included, enhancing model performance.
- XGBoost uses a subset of features when splitting tree nodes, similar to random forests, to improve diversity among base learners.
- XGBoost employs a second-order approximation in its boosting algorithm, incorporating the Hessian of the loss along with the gradient for more accurate predictions.
- XGBoost is designed to prevent the ensemble from fixating too much on a few datapoints, thereby lowering the overall variance and improving model robustness.
